# Display Functions

> Functions to format and display the discovered information:

In [ ]:
#| default_exp cli.display

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Any, Optional
import signal
import sys
import errno
from cjm_fasthtml_tailwind.cli.utils import (
    SearchResult, discover_utility_modules,
    load_code_from_file, display_items_generic, handle_module_not_found,
    find_usage_in_items, get_view_command, format_usage_examples, simple_item_formatter,
    indented_item_formatter, list_utility_modules
)
from cjm_fasthtml_tailwind.cli.factory_extraction import (
    list_all_factories, list_module_factories, get_factory_by_name
)
from cjm_fasthtml_tailwind.cli.example_discovery import (
    list_all_examples, get_example_by_name, list_module_examples
)
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers
from cjm_fasthtml_tailwind.cli.search import search_all
from cjm_fasthtml_tailwind.cli.core_utils_discovery import get_core_utilities
from cjm_fasthtml_tailwind.cli.test_code import execute_test_code
from cjm_fasthtml_tailwind.cli.imports import get_recommended_imports
from cjm_fasthtml_tailwind.cli.pattern_scanner import (
    analyze_and_suggest_input, display_input_analysis, 
    InputType, detect_input_type
)
from cjm_fasthtml_tailwind.cli.cli_config import LibraryConfig, get_active_config

In [ ]:
#| export
# Handle broken pipe errors gracefully when output is piped
def _handle_broken_pipe():
    """Set up signal handlers to gracefully handle broken pipes."""
    # Ignore SIGPIPE on Unix-like systems
    try:
        signal.signal(signal.SIGPIPE, signal.SIG_DFL)
    except AttributeError:
        # SIGPIPE doesn't exist on Windows
        pass

# Call the handler on module import
_handle_broken_pipe()

In [ ]:
#| export
def safe_print(*args, **kwargs):
    """Print with broken pipe error handling.
    
    This wrapper function catches BrokenPipeError and IOError exceptions
    that occur when output is piped to commands like head, tail, etc.
    """
    try:
        print(*args, **kwargs)
    except BrokenPipeError:
        # Exit cleanly when pipe is broken
        sys.exit(0)
    except IOError as e:
        if e.errno == errno.EPIPE:  # Broken pipe error
            sys.exit(0)
        raise  # Re-raise other IO errors

In [ ]:
#| export
def display_modules(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display all available utility modules with their documentation."""
    if config is None:
        config = get_active_config()
    
    modules = list_utility_modules(config)
    
    # Convert dict to list of tuples for the formatter
    module_list = [(name, doc) for name, doc in modules.items()]
    
    # Create formatter
    def module_formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        name, doc = item
        return f"{name}: {doc}"
    
    # Instructions
    instructions = []
    if modules:
        first_module = list(modules.keys())[0]
        instructions.extend([
            ("To explore a specific module:", f"{config.cli_command} factories -m <module>"),
            ("Example:", f"{config.cli_command} factories -m {first_module}")
        ])
    
    display_items_generic(
        items=module_list,
        title="Available Utility Modules:",
        item_formatter=module_formatter,
        item_type="modules",
        instructions=instructions
    )

In [ ]:
#| export
def display_module_factories(
    module_name: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display all factories in a specific module."""
    if config is None:
        config = get_active_config()
        
    factories = list_module_factories(module_name)
    
    if not factories:
        handle_module_not_found("factories", module_name)
        return
    
    print(f"Factories in '{module_name}' module:")
    print("=" * 60, "\n")

    for factory_info in factories:
        print(f"{factory_info.name}: {factory_info.doc}")
    
    print(f"\nTotal factories: {len(factories)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View detailed factory info:")
    if factories:
        first_factory = factories[0].name
        print(f"    {config.cli_command} factory {module_name} {first_factory}")
    
    print(f"\n  View usage examples:")
    print(f"    {config.cli_command} examples -m {module_name}")
    
    print(f"\n  View helper functions:")
    print(f"    {config.cli_command} helpers -m {module_name}")
    
    print(f"\n  Search within this module:")
    print(f"    {config.cli_command} search <query> --in factories")

In [ ]:
#| export
def display_all_factories(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display all factories across all modules."""
    if config is None:
        config = get_active_config()
        
    all_factories = list_all_factories()
    
    # Create formatter using simple_item_formatter
    factory_formatter = simple_item_formatter('name', 'doc')
    indented_formatter = indented_item_formatter("  ")
    
    # Instructions
    instructions = []
    if all_factories:
        first_module = list(all_factories.keys())[0]
        first_factory = all_factories[first_module][0].name if all_factories[first_module] else None
        
        instructions.extend([
            ("View factories for a specific module:", f"{config.cli_command} factories -m {first_module}" if first_module else None),
            ("View detailed factory info:", f"{config.cli_command} factory {first_module} {first_factory}" if first_module and first_factory else None),
            ("Search for specific factories:", f"{config.cli_command} search <query> --in factories")
        ])
    
    display_items_generic(
        items=all_factories,
        title="All Factories by Module:",
        item_formatter=indented_formatter(lambda item: f"{item.name} - {item.doc}"),
        item_type="factories",
        instructions=instructions
    )

In [ ]:
#| export
def display_module_examples(
    module_name: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display all usage examples in a specific module."""
    if config is None:
        config = get_active_config()
        
    examples = list_module_examples(module_name)
    
    if not examples:
        handle_module_not_found("examples", module_name)
        return

    # Create formatter
    def example_formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        return f"  - {item.feature}: {item.docstring}\n    View with: {get_view_command('example', item.module_name, item.feature)}\n"
    
    display_items_generic(
        items=examples,
        title=f"Usage Examples in '{module_name}' module:",
        item_formatter=example_formatter,
        item_type="example functions"
    )
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View the source code for a specific example:")
    if examples:
        first_example = examples[0]
        print(f"    {config.cli_command} example {module_name} {first_example.feature}")
    
    print(f"\n  View factories in this module:")
    print(f"    {config.cli_command} factories -m {module_name}")
    
    print(f"\n  Search for specific patterns in examples:")
    print(f"    {config.cli_command} search <query> --in examples --include-source")

In [ ]:
#| export
def display_all_examples(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display all usage examples across all modules."""
    if config is None:
        config = get_active_config()
    
    all_examples = list_all_examples()
    
    # Create formatter
    def example_formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        return f"  - {item.feature}: {item.docstring}"
    
    display_items_generic(
        items=all_examples,
        title="All Usage Examples by Module:",
        item_formatter=example_formatter,
        item_type="example functions"
    )

    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View examples for a specific module:")
    if all_examples:
        first_key = list(all_examples.keys())[0]
        first_example = all_examples[first_key][0]
        print(f"    {config.cli_command} examples -m {first_example.module_name}")
    print(f"  View the source code for a specific example:")
    if all_examples:
        first_key = list(all_examples.keys())[0]
        first_example = all_examples[first_key][0]
        print(f"    {config.cli_command} example {first_example.module_name} {first_example.feature}")

In [ ]:
#| export
def display_example_source(
    module_name: str,  # TODO: Add description
    feature: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display the source code of a specific example function."""
    if config is None:
        config = get_active_config()
        
    example = get_example_by_name(module_name, feature)
    
    if not example:
        print(f"No example found for feature '{feature}' in module '{module_name}'.")
        print(f"\nTry running '{config.cli_command} examples --module {module_name}' to see available features.")
        return
    
    print(f"Example: {example.feature.replace('_', ' ').title()} ({example.module_name} module)")
    print(f"Function: {example.name}")
    print("=" * 60)
    print(example.source)

In [ ]:
#| export
def display_module_helpers(
    module_name: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display helper functions available in a specific module."""
    if config is None:
        config = get_active_config()
        
    helpers = get_module_helpers(module_name)
    
    if not helpers:
        handle_module_not_found("helper functions", module_name)
        return
    
    print(f"Helper Functions in '{module_name}' module:")
    print("=" * 60)
    
    for helper_info in helpers:
        print(f"\n{helper_info.name}{helper_info.signature}")
        print("-" * 40)
        print(helper_info.docstring)
    
    print(f"\nTotal helper functions: {len(helpers)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View helper source code:")
    if helpers:
        first_helper = helpers[0].name
        print(f"    {config.cli_command} helper {module_name} {first_helper}")
    
    print(f"\n  View factories in this module:")
    print(f"    {config.cli_command} factories -m {module_name}")
    
    print(f"\n  View usage examples:")
    print(f"    {config.cli_command} examples -m {module_name}")
    
    print(f"\n  Search for helper usage:")
    if helpers:
        print(f"    {config.cli_command} search {helpers[0].name} --include-source")

In [ ]:
#| export
def display_helper_source(
    module_name: str,  # TODO: Add description
    helper_name: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display the source code of a specific helper function."""
    if config is None:
        config = get_active_config()
        
    helpers = get_module_helpers(module_name)
    
    helper_info = None
    for h in helpers:
        if h.name == helper_name:
            helper_info = h
            break
    
    if not helper_info:
        print(f"No helper function '{helper_name}' found in module '{module_name}'.")
        print(f"\nTry running '{config.cli_command} helpers --module {module_name}' to see available helpers.")
        return
    
    print(f"Helper: {helper_info.name} ({helper_info.module_name} module)")
    print(f"Signature: {helper_info.name}{helper_info.signature}")
    print("=" * 60)
    print(helper_info.source)
    
    # Find test examples that demonstrate this helper
    examples = list_module_examples(module_name)
    if examples:
        # Look for the helper examples test function
        helper_example = None
        for example in examples:
            if example.feature == 'helper' or example.feature == 'helpers':
                helper_example = example
                break
        
        if helper_example:
            # Check if this specific helper is used in the test
            import re
            patterns = [
                rf'\b{helper_name}\(',           # helper_name(
                rf'\bstr\({helper_name}\b',      # str(helper_name
                rf'\({helper_name}\)',           # (helper_name)
            ]
            
            if any(re.search(pattern, helper_example.source) for pattern in patterns):
                print(f"\nTest Example:")
                print(f"  {helper_example.name} ({helper_example.docstring})")
                print(f"  View with: {config.cli_command} example {module_name} {helper_example.feature}")
            else:
                print(f"\nNo test example found that demonstrates the '{helper_name}' helper.")
        else:
            print(f"\nNo helper examples test function found in module '{module_name}'.")

In [ ]:
#| export
def display_all_helpers(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display all helper functions across all modules."""
    if config is None:
        config = get_active_config()
        
    all_helpers = {}
    
    for module_name, module in discover_utility_modules(config):
        helpers = get_module_helpers(module_name)
        if helpers:
            all_helpers[module_name] = helpers
    
    # Create formatter
    def helper_formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        # Get first line of docstring for concise display
        first_line = item.docstring.split('\n')[0] if item.docstring else "No documentation available"
        return f"  - {item.name}: {first_line}"
    
    display_items_generic(
        items=all_helpers,
        title="All Helper Functions by Module:",
        item_formatter=helper_formatter,
        item_type="helper functions"
    )

In [ ]:
#| export
def display_factory_info(
    module_name: str,  # TODO: Add description
    factory_name: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display detailed information about a specific factory."""
    if config is None:
        config = get_active_config()
        
    factory_info = get_factory_by_name(module_name, factory_name)
    
    if not factory_info:
        print(f"No factory '{factory_name}' found in module '{module_name}'.")
        print(f"\nTry running '{config.cli_command} factories --module {module_name}' to see available factories.")
        return
    
    print(f"Factory: {factory_info.name} ({factory_info.module_name} module)")
    print("=" * 60)
    
    # Get detailed info from the factory
    try:
        info = factory_info.factory.get_info()
        
        # Display description
        print(f"\nDescription:")
        print(f"  {info['description']}")
        
        # Display valid inputs
        print(f"\nValid Inputs:")
        if isinstance(info['valid_inputs'], list):
            for input_desc in info['valid_inputs']:
                print(f"  - {input_desc}")
        else:
            print(f"  {info['valid_inputs']}")
        
        # Display options
        if 'options' in info:
            print(f"\nOptions:")
            if isinstance(info['options'], dict):
                for key, value in info['options'].items():
                    if isinstance(value, dict):
                        print(f"  {key}:")
                        for sub_key, sub_value in value.items():
                            print(f"    - {sub_key}: {sub_value}")
                    else:
                        print(f"  - {key}: {value}")
            else:
                print(f"  {info['options']}")
            
    except AttributeError:
        print("\nDetailed information not available for this factory.")
        print("Factory type:", type(factory_info.factory).__name__)
    
    # Find test examples that use this factory
    all_examples = list_all_examples()
    example_usage = find_usage_in_items(
        target_name=factory_name,
        items=all_examples,
        source_getter=lambda e: e.source,
        item_type="examples"
    )
    
    if example_usage:
        print(f"\nTest Examples Using This Factory:")
        formatted = format_usage_examples(
            usage_items=example_usage,
            item_name_getter=lambda e: e.name,
            item_type="examples",
            view_command_type="example"
        )
        for line in formatted:
            print(line)
    else:
        print(f"\nNo test examples found that specifically use the '{factory_name}' factory.")
    
    # Find helper functions that use this factory
    all_helpers = {}
    for mod_name, module in discover_utility_modules(config):
        helpers = get_module_helpers(mod_name)
        if helpers:
            all_helpers[mod_name] = helpers
    
    helper_usage = find_usage_in_items(
        target_name=factory_name,
        items=all_helpers,
        source_getter=lambda h: h.source,
        item_type="helpers"
    )
    
    if helper_usage:
        print(f"\nHelper Functions Using This Factory:")
        for mod_name, helper in helper_usage:
            # Get first line of docstring for concise display
            first_line = helper.docstring.split('\n')[0] if helper.docstring else "No description available"
            print(f"  - {helper.name}: {first_line}")
            print(f"    View with: {get_view_command('helper', mod_name, helper.name)}")
    else:
        print(f"\nNo helper functions found that specifically use the '{factory_name}' factory.")

In [ ]:
#| export
def display_search_results(
    results: List[SearchResult],  # TODO: Add description
    query: str  # TODO: Add description
): # TODO: Add type hint
    """Display search results in a formatted way."""
    if not results:
        print(f"No results found for '{query}'")
        return
    
    try:
        print(f"Search Results for '{query}':")
        print("=" * 60)
        
        # Group results by content type
        by_type = {}
        for result in results:
            if result.content_type not in by_type:
                by_type[result.content_type] = []
            by_type[result.content_type].append(result)
        
        # Display results by type
        for content_type, type_results in by_type.items():
            print(f"\n{content_type.title()}s ({len(type_results)} results)")
            print("-" * 40)
            
            for result in type_results:
                print(f"\n  {result.item_name} ({result.module_name} module)")
                print(f"  Found in: {result.match_location}")
                print(f"  Context: {result.match_context}")
                
                # Show how to view the full item
                if result.content_type == 'factory':
                    view_cmd = get_view_command('factory', result.module_name, result.item_name)
                elif result.content_type == 'example':
                    # Extract feature from name
                    feature = result.item_name.replace(f'test_{result.module_name}_', '').replace('_examples', '')
                    view_cmd = get_view_command('example', result.module_name, feature)
                elif result.content_type == 'helper':
                    view_cmd = get_view_command('helper', result.module_name, result.item_name)
                elif result.content_type == 'module':
                    view_cmd = get_view_command('module', result.module_name, result.module_name)
                else:
                    view_cmd = ""
                
                if view_cmd:
                    print(f"  View with: {view_cmd}")
        
        print(f"\nTotal results: {len(results)}")
    except BrokenPipeError:
        # Gracefully handle broken pipes when output is piped
        # Exit cleanly without error message
        sys.exit(0)
    except IOError as e:
        # Handle other IO errors (e.g., when stdout is closed)
        if e.errno == errno.EPIPE:  # Broken pipe
            sys.exit(0)
        raise  # Re-raise other IO errors

In [ ]:
#| export
def display_core_utility_source(
    util_name: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display the source code of a specific core utility function."""
    if config is None:
        config = get_active_config()
        
    utilities = get_core_utilities(config)
    
    util_info = None
    for u in utilities:
        if u.name == util_name:
            util_info = u
            break
    
    if not util_info:
        print(f"No core utility '{util_name}' found.")
        print(f"\nTry running '{config.cli_command} core-utils' to see available utilities.")
        return
    
    print(f"Core Utility: {util_info.name}")
    print(f"Module: {util_info.module}")
    print(f"Signature: {util_info.name}{util_info.signature}")
    print("=" * 60)
    print(util_info.source)
    
    # Find examples that use this utility
    print(f"\nFinding usage examples...")
    
    # Search in all test examples
    all_examples = list_all_examples()
    example_usage = find_usage_in_items(
        target_name=util_name,
        items=all_examples,
        source_getter=lambda e: e.source,
        item_type="examples"
    )
    
    if example_usage:
        print(f"\nFound in {len(example_usage)} test examples:")
        formatted = format_usage_examples(
            usage_items=example_usage,
            item_name_getter=lambda e: e.name,
            item_type="examples",
            view_command_type="example"
        )
        for line in formatted:
            print(line)
    
    # Search in helper functions
    all_helpers = {}
    for module_name, module in discover_utility_modules(config):
        helpers = get_module_helpers(module_name)
        if helpers:
            all_helpers[module_name] = helpers
    
    helper_usage = find_usage_in_items(
        target_name=util_name,
        items=all_helpers,
        source_getter=lambda h: h.source,
        item_type="helpers"
    )
    
    if helper_usage:
        print(f"\nFound in {len(helper_usage)} helper functions:")
        for module_name, helper in helper_usage:
            print(f"  - {helper.name} ({module_name} module)")
            print(f"    View with: {get_view_command('helper', module_name, helper.name)}")
    
    # Suggest search for more examples
    print(f"\nTo find more usage examples:")
    print(f"  {config.cli_command} search {util_name} --include-source")

In [ ]:
#| export
def display_core_utilities(
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display all core utility functions."""
    if config is None:
        config = get_active_config()
        
    utilities = get_core_utilities(config)
    
    if not utilities:
        print("No core utilities found.")
        return
    
    print("Core Utility Functions:")
    print("=" * 60)
    
    for util in utilities:
        print(f"\n{util.name}{util.signature}")
        print("-" * 40)
        print(f"Module: {util.module}")
        print(f"Import: {util.import_statement}")
        print(f"\nDocumentation:")
        print(util.docstring)
    
    print(f"\nTotal core utilities: {len(utilities)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    if utilities:
        first_util = utilities[0]
        print(f"  View source code:")
        print(f"    {config.cli_command} core-util {first_util.name}")
    print(f"\n  Search for usage examples:")
    print(f"    {config.cli_command} search combine_classes --include-source")

In [ ]:
#| export
def display_imports(
    modules: Optional[List[str]] = None,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display recommended import statements."""
    if config is None:
        config = get_active_config()
        
    imports = get_recommended_imports(modules)
    
    if modules:
        print(f"Recommended imports for modules: {', '.join(modules)}")
    else:
        print("Recommended imports for all modules:")
    print("=" * 60)
    print()
    
    for imp in imports:
        print(imp)
    
    print("\nUsage:")
    print("- Copy these imports to the top of your Python file")
    print("- The imports include all factories, helpers, and core utilities")
    print("- FastHTML components are available via 'from fasthtml.common import *'")
    
    if not modules:
        print("\nTo get imports for specific modules only:")
        print(f"  {config.cli_command} imports --modules spacing sizing")

In [ ]:
#| export
def display_test_code_result(
    success: bool,  # TODO: Add description
    stdout: str,  # TODO: Add description
    stderr: str,  # TODO: Add description
    code: str,  # TODO: Add description
    config: Optional[LibraryConfig] = None  # Optional configuration to use
): # TODO: Add type hint
    """Display the results of test code execution."""
    if config is None:
        config = get_active_config()
        
    print("Test Code Execution Result:")
    print("=" * 60)
    
    # Show the code that was tested
    print("\nCode tested:")
    print("-" * 40)
    print(code)
    print("-" * 40)
    
    # Show the result
    if success:
        print("\n✅ Code executed successfully!")
        if stdout:
            print("\nOutput:")
            print(stdout)
    else:
        print("\n❌ Code execution failed!")
        if stderr:
            print("\nError:")
            print(stderr)
    
    # Show helpful tips
    if not success:
        print("\nTips:")
        print(f"- Check that all factory names are correct (use '{config.cli_command} factories')")
        print(f"- Verify function signatures (use '{config.cli_command} helper <module> <name>')")
        print(f"- Look at examples for usage patterns (use '{config.cli_command} examples')")
        print(f"- Search for specific patterns (use '{config.cli_command} search <query>')")

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()